In [1]:
import numpy as np
import numpy.polynomial as P
import scipy as sp
from matplotlib import pyplot as plt
from tqdm import tqdm
#from sklearn.preprocessing import PolynomialFeatures
from multiprocessing import Pool
import multiprocessing
import ZVnbrosse
from potentials import PotentialOne, PotentialTwo, PotentialOnePrime
from zv_cv import Eval_ZVCV
from samplers import MCMC_sampler,Generate_train,ULA_light
from baselines import set_function,construct_ESVM_kernel,GenerateSigma
from martingale import approx_q,test_traj
from optimize import Run_eval_test,optimize_parallel_new 
from utils import *
import copy

In [2]:
*10**4 # Burn in period
N_train = 2*10**5 # Number of samples on which we optimize
N_test = 2*10**3 # Number of samples
step = 0.05 # Step size
#put 0.5 for MALA
#step = 0.2
n_traj = 24 # Number of independent MCMC trajectories for test
f_type = "sum"

Choose density parameters

In [3]:
d = 2
B = np.pi/2
#M = 10.0
#Cur_pot = PotentialOne(B,M,d)
mu =0.5
sigma = 0.5
M = 1.0
Cur_pot = PotentialTwo(M,mu,sigma,d)

### Generate data

In [4]:
sampler = {"sampler":"MALA","burn_type":"full","main_type":"full"} # Sampling method

if sampler["sampler"] == "ULA":
    res = Generate_train(n_traj, sampler, Cur_pot, step, N_burn, N_train, d)
    res = np.asarray(res)
    traj,traj_grad = res[:,0,:,:],res[:,1,:,:]
else:
    res = Generate_train(n_traj, sampler, Cur_pot, step, N_burn, N_train, d)
    traj = []
    traj_grad = []
    for i in range(len(res)):
        traj.append(res[i][0])
        traj_grad.append(res[i][1])
        print("accepted = ",res[i][2])
    traj = np.asarray(traj)
    traj_grad = np.asarray(traj_grad)

ncores =  12


Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-2:
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run

KeyboardInterrupt
Process ForkPoolWorker-11:
Process ForkPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
Process ForkPoolWorker-9:
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/svsamsonov/anaconda3/lib/python3

Traceback (most recent call last):
  File "/home/svsamsonov/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-b1368439a00c>", line 8, in <module>
    res = Generate_train(n_traj, sampler, Cur_pot, step, N_burn, N_train, d)
  File "/home/svsamsonov/Research/VR-MCMC/Code/samplers.py", line 288, in Generate_train
    res = trav.starmap(MALA, [(start_seed+i, Potential, step,N_burn,N_gen,d,burn_type,main_type) for i in range (n_traj)])
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/pool.py", line 276, in starmap
    return self._map_async(func, iterable, starmapstar, chunksize).get()
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/pool.py", line 651, in get
    self.wait(timeout)
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/pool.py", line 648, in wait
    self._event.wait(timeout)
  File "/home/svsamsonov/anaconda

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args,

  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/home/svsamsonov/anaconda3/lib/python3.7/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "

In [ ]:
print(traj.shape)
print(traj_grad.shape)
traj_grad = (-1)*traj_grad

In [ ]:
test_seed = 1453
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(Eval_ZVCV, [(traj[i,:,:],traj_grad[i,:,:],f_type) for i in range (n_traj)])
trav.close()

In [ ]:
res_arr = np.asarray(res)
print(res_arr.shape)

### Comparison plots

In [ ]:
title = ""
labels = ['Vanilla\n MALA', 'MALA \nwith ZV-1', 'MALA \nwith CV-1']

In [ ]:
data = [res_arr[:,0],res_arr[:,1],res_arr[:,3]] 
boxplot_ind(data, title, labels)

In [ ]:
title = ""
labels = ['ULA \nwith ZV-1', 'ULA \nwith CV-1']

In [ ]:
data = [res_arr[:,1],res_arr[:,3]] 
boxplot_ind(data, title, labels)

In [ ]:
title = ""
labels = ['Vanilla\n ULA', 'ULA \nwith ZV-2', 'ULA \nwith CV-2']

In [ ]:
data = [res_arr[:,0],res_arr[:,2],res_arr[:,4]] 
boxplot_ind(data, title, labels)

In [ ]:
title = ""
labels = ['ULA \nwith ZV-2', 'ULA \nwith CV-2']

In [ ]:
data = [res_arr[:,2],res_arr[:,4]] 
boxplot_ind(data, title, labels)